In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
import os
import time
import ast
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils import shuffle
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt


In [52]:
import pandas as pd
folder_path = "/content/drive/MyDrive/f1_tenth/"

In [53]:
import os
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]


In [54]:
import pandas as pd
all_df = []
for file in csv_files:
  file_path = os.path.join(folder_path, file)
  df = pd.read_csv(file_path)
  all_df.append(df)
combined_df = pd.concat(all_df, ignore_index=True)


In [55]:
df.head()

,timestamp,speed,steer_angle,lidar_data
0,1.744671e+09,0.545222,0.000000,"[1.749, 1.726, 1.713, 1.69, 1.669, 1.647, 1.62..."
1,1.744671e+09,0.545222,-0.005370,"[1.781, 1.759, 1.735, 1.725, 1.697, 1.678, 1.6..."
2,1.744671e+09,0.545222,-0.008891,"[1.773, 1.746, 1.739, 1.713, 1.687, 1.666, 1.6..."
3,1.744671e+09,0.545222,-0.019456,"[1.756, 1.736, 1.707, 1.687, 1.67, 1.658, 1.63..."
4,1.744671e+09,0.545222,-0.047640,"[1.731, 1.705, 1.686, 1.671, 1.661, 1.62, 1.61..."


In [56]:
def linear_map(x, x_min, x_max, y_min, y_max):
    return (x - x_min) / (x_max - x_min) * (y_max - y_min) + y_min

In [57]:
lidar_list = []
servo_list = []
speed_list = []

In [58]:
for i, row in df.iterrows():
    speed_val = float(row['speed'])
    steer_val = float(row['steer_angle'])

    lidar_str = row['lidar_data']
    lidar_vals = ast.literal_eval(lidar_str)

    lidar_list.append(lidar_vals)
    servo_list.append(steer_val)
    speed_list.append(speed_val)

In [59]:
lidar = np.array(lidar_list, dtype=np.float32)
servo = np.array(servo_list, dtype=np.float32)
speed = np.array(speed_list, dtype=np.float32)

min_speed = np.min(speed)
max_speed = np.max(speed)
speed_mapped = linear_map(speed, min_speed, max_speed, 0, 1)

In [60]:
model_name = 'TLN'
loss_figure_path = './Figures/loss_curve.png'
train_ratio = 0.80
lr = 5e-5
batch_size = 64
num_epochs = 20
hz = 40

In [61]:
#lets shuffle this up for train, test split

num_samples = len(lidar)
indices = np.arange(num_samples)
np.random.shuffle(indices)
lidar = lidar[indices]
servo = servo[indices]
speed_mapped = speed_mapped[indices]

train_size = int(train_ratio * num_samples)
x_train = lidar[:train_size]
y_train_servo = servo[:train_size]
y_train_speed = speed_mapped[:train_size]
x_test = lidar[train_size:]
y_test_servo = servo[train_size:]
y_test_speed = speed_mapped[train_size:]

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU AVAILABLE:", torch.cuda.is_available())

GPU AVAILABLE: True


In [63]:
print(f"Total Samples: {num_samples}")
print(f"Training Samples: {len(x_train)}")
print(f"Testing Samples: {len(x_test)}")
print(f"Min_speed: {min_speed:.3f}, Max_speed: {max_speed:.3f}")

Total Samples: 1701
Training Samples: 1360
Testing Samples: 341
Min_speed: 0.000, Max_speed: 1.374


In [64]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(
    np.column_stack([y_train_servo, y_train_speed]), dtype=torch.float32
)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(
    np.column_stack([y_test_servo, y_test_speed]), dtype=torch.float32
)

In [65]:
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [66]:
num_lidar_range_values = x_train.shape[1]
print("num_lidar_range_values:", num_lidar_range_values)

num_lidar_range_values: 1081


In [67]:
class TLNModel(nn.Module):
    def __init__(self, input_length):
        super(TLNModel, self).__init__()

        #cnns like their model
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=24, kernel_size=10, stride=4)
        self.conv2 = nn.Conv1d(in_channels=24, out_channels=36, kernel_size=8, stride=4)
        self.conv3 = nn.Conv1d(in_channels=36, out_channels=48, kernel_size=4, stride=2)
        self.conv4 = nn.Conv1d(in_channels=48, out_channels=64, kernel_size=3, stride=1)
        self.conv5 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1)

        with torch.no_grad():
            dummy = torch.zeros(1, 1, input_length)
            x = F.relu(self.conv1(dummy))
            x = F.relu(self.conv2(x))
            x = F.relu(self.conv3(x))
            x = F.relu(self.conv4(x))
            x = F.relu(self.conv5(x))
            self.flattened_size = x.numel()

        #fcls
        self.fc1 = nn.Linear(self.flattened_size, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 2)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.tanh(self.fc4(x))
        return x

model = TLNModel(num_lidar_range_values).to(device)
print(model)

TLNModel(
  (conv1): Conv1d(1, 24, kernel_size=(10,), stride=(4,))
  (conv2): Conv1d(24, 36, kernel_size=(8,), stride=(4,))
  (conv3): Conv1d(36, 48, kernel_size=(4,), stride=(2,))
  (conv4): Conv1d(48, 64, kernel_size=(3,), stride=(1,))
  (conv5): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=1792, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=True)
  (tanh): Tanh()
)


In [68]:
def huber_loss_np(y_true, y_pred, delta=1.0):
    #im just doing it the way they do it, technically i can just use torch's huber function but this works too
    error = np.abs(y_true - y_pred)
    loss = np.where(error <= delta, 0.5 * error**2, delta * (error - 0.5 * delta))
    return np.mean(loss)

In [69]:
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_losses = []
val_losses = []

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_inputs, batch_targets in train_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)

    #val
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_inputs, batch_targets in test_loader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            val_loss += loss.item() * batch_inputs.size(0)
    val_loss /= len(test_loader.dataset)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.6f} - Val Loss: {val_loss:.6f}")

total_train_time = time.time() - start_time
print(f"Training time = {total_train_time:.2f} seconds")


Epoch 1/20 - Train Loss: 0.093910 - Val Loss: 0.091025
Epoch 2/20 - Train Loss: 0.088049 - Val Loss: 0.080146
Epoch 3/20 - Train Loss: 0.067917 - Val Loss: 0.061838
Epoch 4/20 - Train Loss: 0.055876 - Val Loss: 0.046987
Epoch 5/20 - Train Loss: 0.038766 - Val Loss: 0.024689
Epoch 6/20 - Train Loss: 0.020619 - Val Loss: 0.017022
Epoch 7/20 - Train Loss: 0.016561 - Val Loss: 0.016473
Epoch 8/20 - Train Loss: 0.015233 - Val Loss: 0.015460
Epoch 9/20 - Train Loss: 0.014303 - Val Loss: 0.015022
Epoch 10/20 - Train Loss: 0.013565 - Val Loss: 0.014765
Epoch 11/20 - Train Loss: 0.012969 - Val Loss: 0.014701
Epoch 12/20 - Train Loss: 0.012598 - Val Loss: 0.014260
Epoch 13/20 - Train Loss: 0.012270 - Val Loss: 0.014166
Epoch 14/20 - Train Loss: 0.011903 - Val Loss: 0.014018
Epoch 15/20 - Train Loss: 0.011640 - Val Loss: 0.013934
Epoch 16/20 - Train Loss: 0.011591 - Val Loss: 0.013876
Epoch 17/20 - Train Loss: 0.011344 - Val Loss: 0.013683
Epoch 18/20 - Train Loss: 0.011229 - Val Loss: 0.013594
E

In [70]:
model.eval()
test_loss = 0.0
all_preds = []
with torch.no_grad():
    for batch_inputs, batch_targets in test_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        test_loss += loss.item() * batch_inputs.size(0)
        all_preds.append(outputs.cpu().numpy())

test_loss /= len(test_loader.dataset)
all_preds = np.concatenate(all_preds, axis=0)  # shape: (N_test, 2)

# Print aggregated results
print(f"Overall Test Loss (SmoothL1) = {test_loss:.6f}")
overall_huber = huber_loss_np(y_test_tensor.numpy(), all_preds)
print(f"Overall Huber Loss (NumPy) = {overall_huber:.6f}")

# Evaluate speed vs. servo separately
servo_pred = all_preds[:, 0]
speed_pred = all_preds[:, 1]
servo_hl = huber_loss_np(y_test_tensor.numpy()[:, 0], servo_pred)
speed_hl = huber_loss_np(y_test_tensor.numpy()[:, 1], speed_pred)
print(f"Servo Huber Loss: {servo_hl:.6f}")
print(f"Speed Huber Loss: {speed_hl:.6f}")

Overall Test Loss (SmoothL1) = 0.013397
Overall Huber Loss (NumPy) = 0.013397
Servo Huber Loss: 0.009128
Speed Huber Loss: 0.017667


In [72]:
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_curve.png')
print("Plot saved as loss_curve.png")


Plot saved as loss_curve.png


I'm trying training on first and second csvs only, and testing on third csv with obstacles.

In [73]:
model_1_2 = TLNModel(num_lidar_range_values).to(device)

In [82]:
#========================================================
# 1) LOAD TRAIN AND TEST CSVs SEPARATELY
#========================================================
import ast

def load_csv(filepath):
    df = pd.read_csv(filepath)
    lidar_list, servo_list, speed_list = [], [], []

    for _, row in df.iterrows():
        speed_val = float(row['speed'])
        steer_val = float(row['steer_angle'])
        lidar_vals = ast.literal_eval(row['lidar_data'])

        lidar_list.append(lidar_vals)
        servo_list.append(steer_val)
        speed_list.append(speed_val)

    lidar = np.array(lidar_list, dtype=np.float32)
    servo = np.array(servo_list, dtype=np.float32)
    speed = np.array(speed_list, dtype=np.float32)
    return lidar, servo, speed

folder_path = "/content/drive/MyDrive/f1_tenth/"
train_files = ['ds.csv', 'ds_anti_clock.csv']
test_file = 'ds_obs_anti.csv'

# Load training data from the first two CSVs
lidar_train, servo_train, speed_train = [], [], []
for f in train_files:
    l, s, spd = load_csv(os.path.join(folder_path, f))
    lidar_train.append(l)
    servo_train.append(s)
    speed_train.append(spd)

x_train = np.concatenate(lidar_train, axis=0)
y_train_servo = np.concatenate(servo_train, axis=0)
y_train_speed = np.concatenate(speed_train, axis=0)

# Load test data
x_test, y_test_servo, y_test_speed = load_csv(os.path.join(folder_path, test_file))

# Normalize speed
min_speed = np.min(y_train_speed)
max_speed = np.max(y_train_speed)

y_train_speed = linear_map(y_train_speed, min_speed, max_speed, 0, 1)
y_test_speed = linear_map(y_test_speed, min_speed, max_speed, 0, 1)

# Shuffle training data
x_train, y_train_servo, y_train_speed = shuffle(x_train, y_train_servo, y_train_speed, random_state=42)

print(f"Training Samples: {len(x_train)}")
print(f"Testing Samples: {len(x_test)}")
print(f"Min_speed: {min_speed:.3f}, Max_speed: {max_speed:.3f}")


Training Samples: 5572
Testing Samples: 5171
Min_speed: 0.000, Max_speed: 1.374


In [87]:
#getting the test set (3rd csv with obs)
df_test = pd.read_csv(os.path.join(folder_path, test_file))

lidar_list_test = []
servo_list_test = []
speed_list_test = []

for _, row in df_test.iterrows():
    speed_val = float(row['speed'])
    steer_val = float(row['steer_angle'])
    lidar_vals = ast.literal_eval(row['lidar_data'])

    lidar_list_test.append(lidar_vals)
    servo_list_test.append(steer_val)
    speed_list_test.append(speed_val)

x_test = np.array(lidar_list_test, dtype=np.float32)
y_test_servo = np.array(servo_list_test, dtype=np.float32)
y_test_speed = np.array(speed_list_test, dtype=np.float32)

# Normalize speed using training min/max
y_test_speed = linear_map(y_test_speed, min_speed, max_speed, 0, 1)

# Shuffle test set
x_test, y_test_servo, y_test_speed = shuffle(x_test, y_test_servo, y_test_speed, random_state=42)

# Convert to PyTorch tensors
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(
    np.column_stack([y_test_servo, y_test_speed]), dtype=torch.float32
)


In [88]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(
    np.column_stack([y_train_servo, y_train_speed]), dtype=torch.float32
)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(
    np.column_stack([y_test_servo, y_test_speed]), dtype=torch.float32
)

In [89]:
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [90]:
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_losses = []
val_losses = []

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_inputs, batch_targets in train_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)

    #val
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_inputs, batch_targets in test_loader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            val_loss += loss.item() * batch_inputs.size(0)
    val_loss /= len(test_loader.dataset)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.6f} - Val Loss: {val_loss:.6f}")

total_train_time = time.time() - start_time
print(f"Training time = {total_train_time:.2f} seconds")


Epoch 1/20 - Train Loss: 0.005740 - Val Loss: 0.009942
Epoch 2/20 - Train Loss: 0.005701 - Val Loss: 0.008863
Epoch 3/20 - Train Loss: 0.005547 - Val Loss: 0.009709
Epoch 4/20 - Train Loss: 0.005527 - Val Loss: 0.008941
Epoch 5/20 - Train Loss: 0.005511 - Val Loss: 0.009030
Epoch 6/20 - Train Loss: 0.005416 - Val Loss: 0.009056
Epoch 7/20 - Train Loss: 0.005385 - Val Loss: 0.009962
Epoch 8/20 - Train Loss: 0.005330 - Val Loss: 0.009242
Epoch 9/20 - Train Loss: 0.005262 - Val Loss: 0.009644
Epoch 10/20 - Train Loss: 0.005225 - Val Loss: 0.009353
Epoch 11/20 - Train Loss: 0.005173 - Val Loss: 0.009433
Epoch 12/20 - Train Loss: 0.005132 - Val Loss: 0.010145
Epoch 13/20 - Train Loss: 0.005186 - Val Loss: 0.009239
Epoch 14/20 - Train Loss: 0.005033 - Val Loss: 0.009520
Epoch 15/20 - Train Loss: 0.004989 - Val Loss: 0.009445
Epoch 16/20 - Train Loss: 0.005041 - Val Loss: 0.009439
Epoch 17/20 - Train Loss: 0.004922 - Val Loss: 0.009886
Epoch 18/20 - Train Loss: 0.004888 - Val Loss: 0.009073
E

In [91]:
model.eval()
test_loss = 0.0
all_preds = []
with torch.no_grad():
    for batch_inputs, batch_targets in test_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        test_loss += loss.item() * batch_inputs.size(0)
        all_preds.append(outputs.cpu().numpy())

test_loss /= len(test_loader.dataset)
all_preds = np.concatenate(all_preds, axis=0)  # shape: (N_test, 2)

# Print aggregated results
print(f"Overall Test Loss (SmoothL1) = {test_loss:.6f}")
overall_huber = huber_loss_np(y_test_tensor.numpy(), all_preds)
print(f"Overall Huber Loss (NumPy) = {overall_huber:.6f}")

# Evaluate speed vs. servo separately
servo_pred = all_preds[:, 0]
speed_pred = all_preds[:, 1]
servo_hl = huber_loss_np(y_test_tensor.numpy()[:, 0], servo_pred)
speed_hl = huber_loss_np(y_test_tensor.numpy()[:, 1], speed_pred)
print(f"Servo Huber Loss: {servo_hl:.6f}")
print(f"Speed Huber Loss: {speed_hl:.6f}")

Overall Test Loss (SmoothL1) = 0.009025
Overall Huber Loss (NumPy) = 0.009025
Servo Huber Loss: 0.010334
Speed Huber Loss: 0.007717


In [92]:
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_curve.png')
print("Plot saved as loss_curve.png")


Plot saved as loss_curve.png


Training on all 3 files. 80/20 split

In [108]:
import os
import ast
import numpy as np
import pandas as pd

def linear_map(x, x_min, x_max, y_min, y_max):
    """Map x linearly from [x_min, x_max] to [y_min, y_max]."""
    return (x - x_min) / (x_max - x_min) * (y_max - y_min) + y_min

def load_csv(filepath):
    """Load CSV file and extract lidar, steer angle and speed data."""
    df = pd.read_csv(filepath)
    lidar_list, servo_list, speed_list = [], [], []

    for _, row in df.iterrows():
        speed_val = float(row['speed'])
        steer_val = float(row['steer_angle'])
        lidar_vals = ast.literal_eval(row['lidar_data'])

        lidar_list.append(lidar_vals)
        servo_list.append(steer_val)
        speed_list.append(speed_val)

    lidar = np.array(lidar_list, dtype=np.float32)
    servo = np.array(servo_list, dtype=np.float32)
    speed = np.array(speed_list, dtype=np.float32)
    return lidar, servo, speed

# Define the folder and the list of CSV files to load.
folder_path = "/content/drive/MyDrive/f1_tenth/"
train_files = ['ds.csv', 'ds_anti_clock.csv', 'ds_obs_anti.csv']

# Initialize lists to store data from each file.
lidar_data_list = []
servo_data_list = []
speed_data_list = []

# Load data from each CSV file and append to the lists.
for f in train_files:
    filepath = os.path.join(folder_path, f)
    l, s, spd = load_csv(filepath)
    lidar_data_list.append(l)
    servo_data_list.append(s)
    speed_data_list.append(spd)

# Concatenate data from all CSV files. This produces one array for each variable.
lidar = np.concatenate(lidar_data_list, axis=0)
servo = np.concatenate(servo_data_list, axis=0)
speed = np.concatenate(speed_data_list, axis=0)

# Now compute min and max on the concatenated speed array.
min_speed = np.min(speed)
max_speed = np.max(speed)
speed_mapped = linear_map(speed, min_speed, max_speed, 0, 1)

# Print out some information.
print("Total Samples:", lidar.shape[0])
print(f"Min_speed: {min_speed:.3f}, Max_speed: {max_speed:.3f}")

Total Samples: 10743
Min_speed: 0.000, Max_speed: 1.426


In [109]:
num_samples = len(lidar)
indices = np.arange(num_samples)
np.random.shuffle(indices)
lidar = lidar[indices]
servo = servo[indices]
speed_mapped = speed_mapped[indices]

train_size = int(train_ratio * num_samples)
x_train = lidar[:train_size]
y_train_servo = servo[:train_size]
y_train_speed = speed_mapped[:train_size]
x_test = lidar[train_size:]
y_test_servo = servo[train_size:]
y_test_speed = speed_mapped[train_size:]


In [110]:
print(f"Total Samples: {num_samples}")
print(f"Training Samples: {len(x_train)}")
print(f"Testing Samples: {len(x_test)}")
print(f"Min_speed: {min_speed:.3f}, Max_speed: {max_speed:.3f}")

Total Samples: 10743
Training Samples: 8594
Testing Samples: 2149
Min_speed: 0.000, Max_speed: 1.426


In [111]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)  # shape: (N, 1, LIDAR_LENGTH)
y_train_tensor = torch.tensor(
    np.column_stack([y_train_servo, y_train_speed]), dtype=torch.float32
)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(
    np.column_stack([y_test_servo, y_test_speed]), dtype=torch.float32
)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_lidar_range_values = x_train.shape[1]
print("num_lidar_range_values:", num_lidar_range_values)


num_lidar_range_values: 1081


In [117]:
all_csvs_model = TLNModel(num_lidar_range_values).to(device)
print(all_csvs_model)


TLNModel(
  (conv1): Conv1d(1, 24, kernel_size=(10,), stride=(4,))
  (conv2): Conv1d(24, 36, kernel_size=(8,), stride=(4,))
  (conv3): Conv1d(36, 48, kernel_size=(4,), stride=(2,))
  (conv4): Conv1d(48, 64, kernel_size=(3,), stride=(1,))
  (conv5): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=1792, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=True)
  (tanh): Tanh()
)


In [118]:
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(all_csvs_model.parameters(), lr=lr)

train_losses = []
val_losses = []

start_time = time.time()
for epoch in range(num_epochs):
    all_csvs_model.train()
    running_loss = 0.0
    for batch_inputs, batch_targets in train_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        optimizer.zero_grad()
        outputs = all_csvs_model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)

    #val
    all_csvs_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_inputs, batch_targets in test_loader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = all_csvs_model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            val_loss += loss.item() * batch_inputs.size(0)
    val_loss /= len(test_loader.dataset)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.6f} - Val Loss: {val_loss:.6f}")

total_train_time = time.time() - start_time
print(f"Training time = {total_train_time:.2f} seconds")


Epoch 1/20 - Train Loss: 0.092002 - Val Loss: 0.022106
Epoch 2/20 - Train Loss: 0.014717 - Val Loss: 0.013602
Epoch 3/20 - Train Loss: 0.012875 - Val Loss: 0.012905
Epoch 4/20 - Train Loss: 0.012679 - Val Loss: 0.012480
Epoch 5/20 - Train Loss: 0.012168 - Val Loss: 0.012088
Epoch 6/20 - Train Loss: 0.011257 - Val Loss: 0.010507
Epoch 7/20 - Train Loss: 0.009563 - Val Loss: 0.009132
Epoch 8/20 - Train Loss: 0.008738 - Val Loss: 0.008767
Epoch 9/20 - Train Loss: 0.008332 - Val Loss: 0.008444
Epoch 10/20 - Train Loss: 0.008070 - Val Loss: 0.008194
Epoch 11/20 - Train Loss: 0.007797 - Val Loss: 0.007943
Epoch 12/20 - Train Loss: 0.007591 - Val Loss: 0.007589
Epoch 13/20 - Train Loss: 0.007452 - Val Loss: 0.007488
Epoch 14/20 - Train Loss: 0.007312 - Val Loss: 0.007348
Epoch 15/20 - Train Loss: 0.007237 - Val Loss: 0.007296
Epoch 16/20 - Train Loss: 0.007110 - Val Loss: 0.007181
Epoch 17/20 - Train Loss: 0.007038 - Val Loss: 0.007155
Epoch 18/20 - Train Loss: 0.006937 - Val Loss: 0.007286
E

In [119]:
model.eval()
test_loss = 0.0
all_preds = []
with torch.no_grad():
    for batch_inputs, batch_targets in test_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        test_loss += loss.item() * batch_inputs.size(0)
        all_preds.append(outputs.cpu().numpy())

test_loss /= len(test_loader.dataset)
all_preds = np.concatenate(all_preds, axis=0)  # shape: (N_test, 2)

# Print aggregated results
print(f"Overall Test Loss (SmoothL1) = {test_loss:.6f}")
overall_huber = huber_loss_np(y_test_tensor.numpy(), all_preds)
print(f"Overall Huber Loss (NumPy) = {overall_huber:.6f}")

# Evaluate speed vs. servo separately
servo_pred = all_preds[:, 0]
speed_pred = all_preds[:, 1]
servo_hl = huber_loss_np(y_test_tensor.numpy()[:, 0], servo_pred)
speed_hl = huber_loss_np(y_test_tensor.numpy()[:, 1], speed_pred)
print(f"Servo Huber Loss: {servo_hl:.6f}")
print(f"Speed Huber Loss: {speed_hl:.6f}")


Overall Test Loss (SmoothL1) = 0.006200
Overall Huber Loss (NumPy) = 0.006200
Servo Huber Loss: 0.006969
Speed Huber Loss: 0.005431


In [120]:
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_all_csvs_curve.png')
